In [1]:
import pandas as pd

from functions import *


Short display of the data.

In [2]:
DF = pd.read_csv("philosophy_data.csv", converters={"tokenized_sentence": pd.eval})

#Smaller dataset
# DF = DF.sample(1000)
# DF.reset_index(drop = True, inplace = True)

print(len(DF))
DF.sample(2)

1000


,school,author,title,tokenized_sentence,num_of_chars,num_of_words,string_sentence
529,german_idealism,Immanuel Kant,critique of pure reason,"[concepts, four, paralogisms, transcendental, ...",111,14,concepts four paralogisms transcendental doctr...
416,aristotle,Aristotle,the complete works of aristotle vol 1,"[mutilated, creatures, drag, wounded, limb, re...",70,9,mutilated creatures drag wounded limb remainde...


Extract the tokenized_sentence DF["tokenized_sentence"] sentences and append them into a python list.

In [3]:
# sentences = []
# sentences = collect_sentences_from_tokenized(sentences, DF["tokenized_sentence"])
# sentences[:5]

Load w2v model, tf_idf, bigram, and trigram models and save them in respective

In [49]:
bigram_range = (2,2)
trigram_range = (3,3)
#Remove those terms that occur less than % in the documents count
min_df = 0.001
#Remove those terms that occur more than % in the documents count
max_df = 0.05

In [50]:
# w2v = load_and_or_save_w2v_model(file_name="glove.42B.300d.txt.word2vec", w2v_type="glove", input_file="glove.42B.300d.txt")
tf_idf_vec = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", stopwords=None, min_df=min_df, max_df=max_df)
tf_idf = tf_idf_vec.fit_transform(DF["string_sentence"].tolist())
# 1-2-gram
bigram_vec = CountVectorizer(token_pattern=r"(?u)\b\w+\b", stopwords=None, min_df=min_df, max_df=max_df, ngram_range=bigram_range)
bigram = bigram_vec.fit_transform(DF["string_sentence"].tolist())
tf_idf_bigram_vec = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b",stopwords=None, min_df=min_df, max_df=max_df, ngram_range=bigram_range)
tf_idf_bigram = tf_idf_bigram_vec.fit_transform(DF["string_sentence"].tolist())
# 1-3-gram
trigram_vec = CountVectorizer(token_pattern=r"(?u)\b\w+\b",stopwords=None, min_df=min_df, max_df=max_df, ngram_range=trigram_range)
trigram = trigram_vec.fit_transform(DF["string_sentence"].tolist())
tf_idf_trigram_vec = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b",stopwords=None, min_df=min_df, max_df=max_df, ngram_range=trigram_range)
tf_idf_trigram = tf_idf_trigram_vec.fit_transform(DF["string_sentence"].tolist())

tf_idf.shape, tf_idf_bigram.shape, tf_idf_trigram.shape, bigram.shape, trigram.shape

((1000, 4702), (1000, 10161), (1000, 9376), (1000, 10161), (1000, 9376))

In [ ]:
# tf_idf_gen.get_feature_names_out()

# print(type(tf_idf[0]))
# print(tf_idf.getrow(0))
# [x for x in tf_idf[:].toarray()[0] if x < 1e-8]

# # To TEST MIN_DIF/MAX_DF
# texts = [
#     "Penny bought bright blue fishes.",
#     "Penny bought bright blue and orange fish.",
#     "The cat ate a fish at the store.",
#     "Penny went to the store. Penny ate a bug. Penny saw a fish.",
#     "It meowed once at the bug, it is still meowing at the bug and the fish",
#     "The cat is at the fish store. The cat is orange. The cat is meowing at the fish.",
#     "Penny is a fish"
# ]
# vec = TfidfVectorizer(min_df=1, max_df=1.0, stop_words='english')
# ble = vec.fit_transform(texts)
# pd.DataFrame(ble.toarray(), columns=vec.get_feature_names())
# print(ble.shape)
# ble_kmeans, ble_labels, ble_centroids = apply_kmeans(2, ble, tolerance=1e-8)
# results = pd.DataFrame()
# results['sentence'] = texts
# results['category'] = ble_labels
# results

Dimension reduction.

In [ ]:
# tf_idf = TruncatedSVD(n_components=100).fit_transform(tf_idf)
# tf_idf_bigram = TruncatedSVD(n_components=100).fit_transform(tf_idf_bigram)
# tf_idf_trigram = TruncatedSVD(n_components=100).fit_transform(tf_idf_trigram)
# bigram = TruncatedSVD(n_components=100).fit_transform(bigram)
# trigram = TruncatedSVD(n_components=100).fit_transform(trigram)
#
# tf_idf.shape, tf_idf_bigram.shape, tf_idf_trigram.shape, bigram.shape, trigram.shape

The sentence_vectorizer function summarizes each vectorized word in a sentence and divides the sum with the amount of words in the sentence.
Doing this will allow us to be able to get whole sentences vectorized values instead of individual words.
Using the function with our model (in this case w2v) sentences.

In [ ]:
# indexes_to_remove_from_df = []
# sentences_vectorized = []
# for index, sentence in enumerate(sentences):
#     temp_variable = sentence_vectorizer(sentence, w2v)
#     #Check if return variable is empty
#     if temp_variable is None:
#         indexes_to_remove_from_df.append(index)
#         continue
#     sentences_vectorized.append(temp_variable)
#     if index % 50000 == 0:
#         print(index)
#
# #Remove rows (found in list) from DF, and reset indexing.
# DF.drop(DF.index[indexes_to_remove_from_df], inplace=True)
# DF.reset_index(drop = True, inplace = True)
# sentences = collect_sentences_from_tokenized([], DF["tokenized_sentence"])
#
# #Should be equal
# len(sentences), len(DF["tokenized_sentence"]), len(sentences_vectorized)

Fetch the number of unique School of Thoughts.

In [ ]:
unique_schools = DF["school"].unique().tolist()
print(unique_schools)
num_clusters = len(unique_schools)

Bar plot how many sentences are from each School of Thought.
(Show the total amount of real assigned SOT on each text.)

In [ ]:
total_dict = dict.fromkeys(unique_schools, 0)
for school in DF["school"]:
    total_dict[school] += 1

plt.bar(range(len(total_dict)), total_dict.values(), align='center')
plt.xticks(range(len(total_dict)), total_dict.keys())
plt.xticks(rotation=50)
plt.xlabel("School of Thought")
plt.ylabel("Count")
plt.title("Original Distribution of Sentences per School of Thought")
plt.show()

Apply kmeans with num_clusters=amount of unique schools in the DF.

In [ ]:
start = time.time()

# kmeans, labels, centroids = apply_kmeans(num_clusters, sentences_vectorized)
tf_idf_kmeans, tf_idf_labels, tf_idf_centroids = apply_kmeans(num_clusters, tf_idf, tolerance=1e-8)
tf_idf_bigram_kmeans, tf_idf_bigram_labels, tf_idf_bigram_centroids = apply_kmeans(num_clusters, tf_idf_bigram,tolerance=1e-8)
tf_idf_trigram_kmeans, tf_idf_trigram_labels, tf_idf_trigram_centroids = apply_kmeans(num_clusters, tf_idf_trigram,tolerance=1e-8)
bigram_kmeans, bigram_labels, bigram_centroids = apply_kmeans(num_clusters, bigram,tolerance=1e-8)
trigram_kmeans, trigram_labels, trigram_centroids = apply_kmeans(num_clusters, trigram,tolerance=1e-8)

end = time.time()
print(end - start)

In [ ]:
#Find top words in each cluster on tf_idf
order_centroids = tf_idf_centroids.argsort()[:, ::-1]
terms = tf_idf_vec.get_feature_names()
for i in range(10):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :5]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Summarize the amount each cluster was assigned to a sentence.

In [ ]:
# label_dict = dict.fromkeys(list(set(labels)), 0)
# for cluster in labels:
#     label_dict[cluster] += 1
#
# plt.bar(range(len(label_dict)), label_dict.values(), align='center')
# plt.xticks(range(len(label_dict)), label_dict.keys())
# plt.xlabel("Cluster")
# plt.ylabel("Count")
# plt.title(f"Clustered sentence")
# plt.show()

Apply scatter plot on kmeans results using TSNE.
Scatter plot using TSNE, PCA and UMAP

In [ ]:
# scatter_plot(type='tsne', labels=labels, values=sentences_vectorized, centroids=centroids)
# scatter_plot(type='pca', labels=labels, values=sentences_vectorized, centroids=centroids)
# scatter_plot(type='umap', labels=labels, values=sentences_vectorized, centroids=centroids)

# tf_idf_reduced = TruncatedSVD(500).fit_transform(tf_idf)
# tf_idf_bigram_reduced = TruncatedSVD(500).fit_transform(tf_idf_bigram)
# tf_idf_trigram_reduced = TruncatedSVD(500).fit_transform(tf_idf_trigram)
#
# #PCA
# scatter_plot(type='pca', labels=tf_idf_labels, values=tf_idf_reduced, centroids=tf_idf_centroids)
# scatter_plot(type='pca', labels=tf_idf_bigram_labels, values=tf_idf_bigram_reduced, centroids=tf_idf_bigram_centroids)
# scatter_plot(type='pca', labels=tf_idf_trigram_labels, values=tf_idf_trigram_reduced, centroids=tf_idf_trigram_centroids)
#
# #UMAP
# scatter_plot(type='umap', labels=tf_idf_labels, values=tf_idf_reduced, centroids=tf_idf_centroids)
# scatter_plot(type='umap', labels=tf_idf_bigram_labels, values=tf_idf_bigram_reduced, centroids=tf_idf_bigram_centroids)
# scatter_plot(type='umap', labels=tf_idf_trigram_labels, values=tf_idf_trigram_reduced, centroids=tf_idf_trigram_centroids)
#
# #TSNE
# scatter_plot(labels=tf_idf_labels, values=tf_idf_reduced, centroids=tf_idf_centroids)
# scatter_plot(labels=tf_idf_bigram_labels, values=tf_idf_bigram_reduced, centroids=tf_idf_bigram_centroids)
# scatter_plot(labels=tf_idf_trigram_labels, values=tf_idf_trigram_reduced, centroids=tf_idf_trigram_centroids)

Combine the kmeans clustering results with the labels.
Put the kmeans result in a dictionary:

In [ ]:
#How many times cluster n was assigned to specific SOT
# cluster_n_assigned(DF=DF, labels=labels, schools=unique_schools, name='glove')
cluster_n_assigned(DF=DF, labels=tf_idf_labels, schools=unique_schools, name='tf_idf')
cluster_n_assigned(DF=DF, labels=tf_idf_bigram_labels, schools=unique_schools, name='tf_idf_bigram')
cluster_n_assigned(DF=DF, labels=tf_idf_trigram_labels, schools=unique_schools, name='tf_idf_trigram')
cluster_n_assigned(DF=DF, labels=bigram_labels, schools=unique_schools, name='bigram')
cluster_n_assigned(DF=DF, labels=trigram_labels, schools=unique_schools, name='trigram')

Make a list and convert all "plato" to 0 and "aristotle" to 1, etc.
This can then be applied with the predicted labels with v_measure.

In [ ]:
original_labels = [unique_schools.index(school) for school in DF["school"]]

Apply v-measure.

In [ ]:
# v_measure = v_measure_score(original_labels, labels)
tf_idf_v_measure = v_measure_score(original_labels, tf_idf_labels)
tf_idf_bigram_v_measure = v_measure_score(original_labels, tf_idf_bigram_labels)
tf_idf_trigram_v_measure = v_measure_score(original_labels, tf_idf_trigram_labels)
bigram_v_measure = v_measure_score(original_labels, bigram_labels)
trigram_v_measure = v_measure_score(original_labels, bigram_labels)

tf_idf_v_measure, tf_idf_bigram_v_measure, tf_idf_trigram_v_measure, bigram_v_measure, trigram_v_measure

Calculate the best v_measure for all different text vectorization methods.
Apply different values for min_df and max_df for them and store it in a dict.
Iterations are decided by min_df range * max_df range.

In [ ]:
v_measure_dict, exceptions_dict = average_v_measure(orig_labels=original_labels, num_clusters=num_clusters,DF=DF, max_df_range=(0.005,0.1,0.005), min_df_range=(0.00001,0.0002,0.00001),n=20*20, bigram_range=bigram_range,trigram_range=trigram_range)

Print v_measure values

In [ ]:
sorted_v_measure_dict = {}
for method, list_of_dicts in v_measure_dict.items():
    sorted_v_measure_dict[method] = sorted(list_of_dicts, key=lambda x: x["v_measure"], reverse=True)

#Prints the best v_measure results of each method.
sorted_v_measure_dict["tf_idf"][0], sorted_v_measure_dict["tf_idf_bigram"][0], sorted_v_measure_dict["tf_idf_trigram"][0], sorted_v_measure_dict["bigram"][0], sorted_v_measure_dict["trigram"][0]
